In [13]:
# Libraries for feature extraction:
import librosa
import pandas as pd
import numpy as np
import os
import pathlib
from scipy.stats import kurtosis
from scipy.stats import skew


In [14]:
#Audio data directory:
audio_dir = './Data/genres/'
test_dir = './Data/test_songs/'
spoty_dir='./Data/spotify_songs/'

In [15]:
#Dictionary for genres label encoding:
genres = {'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 
          'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}

In [16]:
def split_songs(X, window, overlap):
    """
    Function to split a song into multiple songs.
    """
    # Temporary lists to hold results
    temp_X = []

    # Get input song array size
    xshape = X.shape[0]
    chunk = int(xshape*window)
    offset = int(chunk*(1.-overlap))
    
    # Split the song and create new ones
    spsong = [X[i:i+chunk] for i in range(0, xshape - chunk + offset, offset)]
    for s in spsong:
        temp_X.append(s)

    return np.array(temp_X)

In [17]:
#Get selected features from each song using librosa and numpy:
def get_features(y, sr, n_fft = 1024, hop_length = 512):
    # Selected features:
    features = {'centroid': None, 'roloff': None, 'flux': None, 'rmse': None, 'zcr': None, 'chroma':None}
    
    # Using librosa to calculate the features
    features['centroid'] = librosa.feature.spectral_centroid(y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['roloff'] = librosa.feature.spectral_rolloff(y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['zcr'] = librosa.feature.zero_crossing_rate(y, frame_length=n_fft, hop_length=hop_length).ravel()
    features['rmse'] = librosa.feature.rms(y, frame_length=n_fft, hop_length=hop_length).ravel()
    features['flux'] = librosa.onset.onset_strength(y=y, sr=sr).ravel()
    features['chroma'] = librosa.feature.chroma_stft(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    
    # Treatment of MFCC feature
    mfcc = librosa.feature.mfcc(y, n_fft = n_fft, hop_length = hop_length, n_mfcc=13)
    for idx, v_mfcc in enumerate(mfcc):
        features['mfcc_{}'.format(idx)] = v_mfcc.ravel()
        
    # Calculate statistics for each feature:
    def get_moments(descriptors):
        result = {}
        for k, v in descriptors.items():
            result[f'{k}_mean'] = np.mean(v)
            result[f'{k}_std'] = np.std(v)
            result[f'{k}_kurtosis'] = kurtosis(v)
            result[f'{k}_skew'] = skew(v)
        return result
    
    dict_agg_features = get_moments(features)
    
    #Calculating one more feature:
    dict_agg_features['tempo'] = librosa.beat.tempo(y, sr=sr)[0]
    
    return dict_agg_features

In [18]:
def read_process_labelled(src_dir, window=0.2, overlap=0.5, debug = True):
    arr_features = []

    # Read files from the folders
    for x,_ in genres.items():
        folder = src_dir + x
    
        for root, subdirs, files in os.walk(folder):
            for file in files:
                # Read the audio file
                file_name = folder + "/" + file
                signal, sr = librosa.load(file_name)
                signal = signal[:660000]
                
                # Debug process
                if debug:
                    print(f"Reading file: {file_name}")
                    
                #Split songs:
                samples=split_songs(signal,window,overlap)

                # Append the result to the data structure
                for s in samples:
                    features = get_features(s, sr)
                    features['genre'] = genres[x]
                    arr_features.append(features)
    return arr_features

In [8]:
def read_process_unlabelled(src_dir, window=1, overlap=0, debug = True):
    arr_features = []

    for root, subdirs, files in os.walk(src_dir):
        for file in files:
            # Read the audio file
            file_name = src_dir + file
            signal, sr = librosa.load(file_name)
            signal = signal[:660000]

            # Debug process
            if debug:
                print("Reading file: {}".format(file_name))
                
            #Split songs:
            samples=split_songs(signal,window,overlap)

            # Append the result to the data structure
            for s in samples:
                features = get_features(s, sr)
                arr_features.append(features)

    return arr_features

In [ ]:
# Get list of dicts with features and convert to dataframe
features = read_process_songs(audio_dir,debug=True)

In [20]:
df=pd.DataFrame(features)

In [25]:
df.to_csv('./Features/features_augmented.csv', index=False)

In [33]:
features1 = read_process_songs_t1(test_dir,debug=True)
df1=pd.DataFrame(features1)
df1.to_csv('./Features/features_augmented_t1.csv', index=False)

Reading file: ./Data/test_songs/blues/blues.00099.wav
Reading file: ./Data/test_songs/classical/classical.00099.wav
Reading file: ./Data/test_songs/country/country.00099.wav
Reading file: ./Data/test_songs/disco/disco.00099.wav
Reading file: ./Data/test_songs/hiphop/hiphop.00099.wav
Reading file: ./Data/test_songs/jazz/jazz.00099.wav
Reading file: ./Data/test_songs/metal/metal.00099.wav
Reading file: ./Data/test_songs/pop/pop.00099.wav
Reading file: ./Data/test_songs/reggae/reggae.00099.wav
Reading file: ./Data/test_songs/rock/rock.00099.wav


In [34]:
features3 = read_process_songs_t3(test_dir,debug=True)
df3=pd.DataFrame(features3)
df3.to_csv('./Features/features_augmented_t3.csv', index=False)

Reading file: ./Data/test_songs/blues/blues.00099.wav
Reading file: ./Data/test_songs/classical/classical.00099.wav
Reading file: ./Data/test_songs/country/country.00099.wav
Reading file: ./Data/test_songs/disco/disco.00099.wav
Reading file: ./Data/test_songs/hiphop/hiphop.00099.wav
Reading file: ./Data/test_songs/jazz/jazz.00099.wav
Reading file: ./Data/test_songs/metal/metal.00099.wav
Reading file: ./Data/test_songs/pop/pop.00099.wav
Reading file: ./Data/test_songs/reggae/reggae.00099.wav
Reading file: ./Data/test_songs/rock/rock.00099.wav


In [127]:
fsp1=read_process_songs_sp1(spoty_dir)

Reading file: ./Data/spotify_songs/reggae.mp3


In [128]:
dfs1=pd.DataFrame(fsp1)

In [129]:
dfs1.to_csv('./Features/fsp1.csv',index=False)

In [130]:
fsp3=read_process_songs_sp3(spoty_dir)

Reading file: ./Data/spotify_songs/reggae.mp3


In [131]:
dfs3=pd.DataFrame(fsp3)

In [132]:
dfs3.to_csv('./Features/fsp3.csv',index=False)

,centroid_kurtosis,centroid_mean,centroid_skew,centroid_std,chroma_kurtosis,chroma_mean,chroma_skew,chroma_std,flux_kurtosis,flux_mean,...,rmse_std,roloff_kurtosis,roloff_mean,roloff_skew,roloff_std,tempo,zcr_kurtosis,zcr_mean,zcr_skew,zcr_std
0,-0.272445,2292.006108,0.639644,371.370890,-0.459790,0.454361,0.717142,0.264166,6.023544,1.166718,...,0.025612,-0.252210,4324.968886,0.819843,871.146437,117.453835,-0.255651,0.145415,-0.031656,0.029378
1,-0.023791,2244.142916,0.831402,406.412611,-0.143059,0.395680,0.911344,0.276072,5.829755,1.247865,...,0.032220,0.380316,4294.772325,1.082511,868.057488,117.453835,-0.612709,0.140400,0.166721,0.034283
2,-0.624096,2381.707646,0.326672,484.115327,-0.469382,0.410844,0.744096,0.282949,6.671955,1.126775,...,0.042709,-1.168401,4817.718520,0.263824,1174.116147,117.453835,0.659586,0.128221,0.465265,0.032395


In [3]:
#mis pruebas para pipeline:
dire='.Data/test_songs/blues/'
signal, sr = librosa.load(file_name)

In [7]:
signal=signal[:660000]

In [11]:
feats=read_process_unlabelled(spoty_dir,1/3,0,debug=True)
df=pd.DataFrame(feats)

Reading file: ./Data/spotify_songs/blues.00099.wav


In [12]:
df

,centroid_kurtosis,centroid_mean,centroid_skew,centroid_std,chroma_kurtosis,chroma_mean,chroma_skew,chroma_std,flux_kurtosis,flux_mean,...,rmse_std,roloff_kurtosis,roloff_mean,roloff_skew,roloff_std,tempo,zcr_kurtosis,zcr_mean,zcr_skew,zcr_std
0,0.949163,1340.141376,1.078580,404.889141,-1.035797,0.475457,0.297787,0.296755,33.079778,1.060128,...,0.066714,1.259107,2765.063590,0.950681,929.608641,92.285156,-0.416028,0.063277,0.833403,0.037962
1,-0.842783,1511.388015,0.393239,470.671228,-0.689767,0.426770,0.645852,0.287938,20.826028,1.158309,...,0.075188,-0.100544,3107.842138,0.302303,1084.135156,92.285156,-0.151604,0.076821,0.689794,0.041826
2,7.779262,1583.142180,2.038264,527.572782,-0.993955,0.444090,0.469302,0.303186,19.604582,1.143739,...,0.079983,0.964444,3130.627271,0.864083,1173.319918,92.285156,2.991650,0.084121,1.062447,0.044637
